In [ ]:
# Installs
%pip install biopython
%pip install pandas
%pip install numpy
%pip install scikit-learn
%pip install matplotlib
%pip install seaborn

In [2]:
import pandas as pd
from numpy.lib.stride_tricks import sliding_window_view
import numpy as np
from Bio import SeqIO, motifs, AlignIO
from Bio.Align import MultipleSeqAlignment
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq
import os
import random
import time
from IPython.display import Image
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [3]:
# => Utility functions
# Function to read text files of sequences
def read_seq_file(in_path):
    with open(os.path.abspath(os.path.expanduser(in_path)), "r") as f:
        sequences = f.read().splitlines()
    return sequences

# Function to generate list of false sequences
def generate_false_sequences(true_seq):
    false_seq = []
    for seq in true_seq:
        x = list(seq)
        random.shuffle(x)
        false_seq.append(''.join(x))
    return false_seq

# Function to vectorize each sequence, produces 1d flattened vectors
def one_hot_encoder(seq):
    integer_encoded = LabelEncoder().fit(np.array(["A", "C", "G", "T"])).transform(list(seq)).reshape(len(list(seq)), 1)
    return OneHotEncoder(sparse=False, dtype=int, categories=[range(4)]).fit_transform(integer_encoded).flatten()


# Function to convert classification prediction into genomic intervals
def generate_genomic_locations(classes, proba, window_size):
    loc_pairs = []
    classes_len = classes.shape[0]
    counter = 0
    while counter < classes_len: 
        if classes[counter] == 1:
            loc_pairs.append([counter + 1, counter + window_size])
            loc_pairs[-1].extend(proba[counter].tolist())
        counter += 1
    return loc_pairs


# Function to convert locations into GFF format
def generate_gff_df(locations, seqid, strand, score_filter=0.001):
    column_names = ["seqid", "source", "type", "start", "end", "score", "strand", "phase", "attribute"]
    strand_letter = "F" if strand == "+" else "R"
    # small function to generate attributes
    attr_func = lambda row: \
        f"id={row['seqid']}_{strand_letter}_prom_{row.name}" \
        f";name={row['seqid']}_{strand_letter}_prom_{row.name}" \
        f";true_proba={row['true_proba']};false_proba={row['false_proba']}"

    gff_df = pd.DataFrame(locations, columns=["start", "end", "true_proba", "false_proba"])
    gff_df = gff_df[gff_df["true_proba"] <= score_filter].copy()
    gff_df["seqid"] = seqid
    gff_df["source"] = "motif_predictor"
    gff_df["type"] = "motif"
    gff_df["score"] = "."
    gff_df["strand"] = strand
    gff_df["phase"] = "."

    for i in gff_df.index:
        gff_df.at[i, "attribute"] = attr_func(gff_df.loc[i])
    gff_df.drop(["true_proba", "false_proba"], inplace=True, axis=1)
    gff_df = gff_df.reindex(columns=column_names)
    return gff_df

In [4]:
# Inputs

promoters_seq_url = "http://regulondb.ccg.unam.mx/menu/download/datasets/files/PromoterSigma24Set.txt"
genome_file = "./GCF_000005845.2_ASM584v2_genomic.fa"
true_bs_file = "./sigE_binding_sites.txt"
save_dir = "./"
iterations = 500
window_size = 29
threads = 40
score_filter = 0.001

In [14]:
# Load promoter sequences and analyze them
prom_col = ["id", "name", "strand", "tss", "sigma_name", "sequence", "evidence", "confidence"]
promotors_df = pd.read_csv(promoters_seq_url, comment="#", sep="\t", names=prom_col)
promotors_df.dropna(subset = ["sequence"], inplace=True)
promotors_df = promotors_df[promotors_df["confidence"].isin(["Strong", "Confirmed"])]
for x, y in enumerate(promotors_df["sequence"].tolist()):
    print(f">{x}\n{y[: 60].upper()}")

>0
TAGATGTCCTTGATTAACACCAAAATTAAACCTTTTAAAAACCAGGCATTCAAAAACGGC
>1
AAAGAAAATAATTAATTTTACAGCTGTTAAACCAAACGGTTATAACCTGGTCATACGCAG
>2
CTGCTGTTCCTTGCGATCGAAAAGATCAAGGGCGGACCGGTATCCGAGCGGGTTCAAGAC
>3
GCGGAAGCACAAATTGCACCAGGTACGGAACTAAAAGCCGTAGATGGTATCGAAACGCCT
>4
AAATACTTATGGTGCGCTGGCTTCTTTGGAACTTGCGCAGCAATTTGTTGACAAAAATGA
>5
ACGCGTTATTAATCAGCGTCTGATGCCATTACACAACAAACTATTTGTCGAACCCAATCC
>6
CTGATCTAAGCGTTGACCGAGTTGGTTTTCGGACACCGTTGCAGTGAGCTGTACTCGTTG
>7
TTTTATTCGCGAACGTGAATAATCCGGGAACATTTCGGCCAAAGCCTGATCTAAGCGTTG
>8
GGCGAATGCGAAAGAACTGCTTGCAGCGTAAACTTTTTTCCTGCTTCACGGTCAGAGTAA
>9
GCCGTTACACTCAAAGGCGGCGCGGTGGGAACGATATTTCACAGTATCGGTCAAATGACT
>10
TAAAAAATATCTTGTATGTGATCCAGATCACATCTATCATTTAGTTATCGATCGTTAAGT
>11
ATTCTGAAAGTTAAAGGGCGCATGAATGAACTTATGGCGCTTCATACGGGTCAATCATTA
>12
TGGAAAATTCTTAGAAACCGATCACATACAGCTGCATTTATTAAGGTTATCATCCGTTTC
>13
TTTACCTTTTGCAGAAACTTTAGTTCGGAACTTCAGGCTATAAAACGAATCTGAAGAACA
>14
ACCGAGCGTCTGCGGCAACTTCATCAACAGCATCACCCGCGGGCGTGATGTCTGAAAAGA
>15
GGAAAAAAATCTGGCTGAAAATACGTTGAAC

In [12]:
prom_instances = []
for i, prom in enumerate(sorted(promotors_df["sequence"].tolist())):
    prom_instances.append(SeqRecord(Seq(prom[: 60].upper()), id=f"seq_{i}"))

align = MultipleSeqAlignment(prom_instances)
print(align)

Alphabet() alignment with 71 rows and 60 columns
AAAACAAGTTGATTTGATATATTAAATGAACAAATTAATCTTGA...ATT seq_0
AAAATATCTTTATTTTTGGTCATACCGTGGAACAAGTGAAGGCA...GCT seq_1
AAAATGAAATTTTTTTGCACAACCGCAGAACTTTTCCGCAGGGC...AGT seq_2
AAAGAAAATAATTAATTTTACAGCTGTTAAACCAAACGGTTATA...CAG seq_3
AAAGTTGTAATAAGCTTGTCTGAATCGAACTTTTAGCCGCTTTA...TCC seq_4
AAATACTTATGGTGCGCTGGCTTCTTTGGAACTTGCGCAGCAAT...TGA seq_5
AACGGAGCGGCCCAGCTTACCTGCCATTGCACTAAATACTGATA...GGC seq_6
AAGCATACGGGCGATGACAAATGCAAAACTGCCTGATGCGCTAC...GGA seq_7
AAGTGGCGGCGCGTTGAAAGAAATTTCGAACCCTGAGAACTTAA...ACT seq_8
AATACAGACAAAGAGCATCTGCGAAAAATTGCACGCGGGATGTT...CTT seq_9
ACCGAGCGTCTGCGGCAACTTCATCAACAGCATCACCCGCGGGC...AGA seq_10
ACGCAGCAAGAACGCCGAGCTGATTGAAAAGGTTAGAACATCCT...CAA seq_11
ACGCCGTTGCTTACAACAGCAGCGATGTAAAAGACATCACTGCC...AGG seq_12
ACGCGTTATTAATCAGCGTCTGATGCCATTACACAACAAACTAT...TCC seq_13
ATAAAGTCATTTTTTATTCAAATATAAGGAACTTAATATTTAAA...AAT seq_14
ATCCAGTGGTATTATTGGCCATTGAAAGAACCTTTTTACATTAT...AGT seq_15
ATCCGCATTCCTTCGCATCGTAGTGCTCGCATT

In [ ]:
sequences_as_fasta = sequences_as_fasta[: -1]
print(promotors_df.shape[0])
# Slice each sequence at 60 position as this position is the trascription start then make it upper
prom_instances = [Seq(prom[: 60].upper()) for prom in promotors_df["sequence"].tolist()]
prom_motifs = motifs.create(prom_instances)
print(prom_motifs.consensus)
prom_motifs.weblogo(f"{save_dir}/logo.png")
Image(filename=f"{save_dir}/logo.png", width=1000)

In [71]:
# Load input files


save_dir = os.path.abspath(save_dir)
genome_file_parsed = SeqIO.parse(os.path.abspath(genome_file), "fasta")
true_sequences = read_seq_file(true_bs_file)
false_sequences = generate_false_sequences(true_sequences)

In [72]:
# Vectorize training data sets in a parallel mode and make dataframes of it

true_dataset_arr = np.array(list(map(one_hot_encoder, true_sequences)), dtype=np.int32)
false_dataset_arr = np.array(list(map(one_hot_encoder, false_sequences)), dtype=np.int32)
# Concatenate training datasets
full_arr = np.concatenate([true_dataset_arr, false_dataset_arr], axis=0)
labels = ([1] * true_dataset_arr.shape[0]) + ([0] * false_dataset_arr.shape[0])

In [73]:
# Prepare the data for prediction
f_data_to_predict = {}
r_data_to_predict = {}
for seq_rec in genome_file_parsed:
    print(f"==> Preparing: {seq_rec.id}")
    # Vectorize the genomic sequence forward and reverse
    f_chrom_vector = one_hot_encoder(str(seq_rec.seq))
    r_chrom_vector = one_hot_encoder(str(seq_rec.reverse_complement().seq))
    # Convert vectorized genome to array of sliding windows
    f_data_to_predict[seq_rec.id] = sliding_window_view(f_chrom_vector, window_size * 4)[::4]
    r_data_to_predict[seq_rec.id] = sliding_window_view(r_chrom_vector, window_size * 4)[::4]
    del f_chrom_vector, r_chrom_vector # free some memory space

==> Preparing: NC_000913.3


In [87]:
models = {"RandomForest": RandomForestClassifier(n_jobs=threads),
          "GradientBoosting": GradientBoostingClassifier(),
          "AdaBoost": AdaBoostClassifier(),
          "MultiLayerPerceptron": MLPClassifier(max_iter=iterations)}
thresholds = {"RandomForest": 0.1,
              "GradientBoosting": 0.05,
              "AdaBoost": 0.43,
              "MultiLayerPerceptron": 0.005}

for model_name, model in models.items():
    t = time.time()
    # Train
    print(f"==> Training {model_name} model")
    model.fit(full_arr, labels)
    save_df = pd.DataFrame()
    for seqid in f_data_to_predict.keys():
        print(f"===> Predicting for: {seqid} using {model_name} model")        
        # Get classification probabilities
        f_proba = model.predict_proba(f_data_to_predict[seqid])
        r_proba = model.predict_proba(r_data_to_predict[seqid])
        # Get classes from propabilities
        f_predict_classes = np.array([model.classes_[i] for i in np.argmax(f_proba, axis=1)])
        r_predict_classes = np.array([model.classes_[i] for i in np.argmax(r_proba, axis=1)])
        # Make genomic coordenates from the classification result
        f_locations = generate_genomic_locations(f_predict_classes, f_proba, window_size)
         # Filp the array up down for reverse strand
        r_locations = generate_genomic_locations(np.flipud(r_predict_classes), np.flipud(r_proba), window_size)
        # Convert to GFF format
        f_gff = generate_gff_df(f_locations, seq_rec.id, "+", thresholds[model_name])
        r_gff = generate_gff_df(r_locations, seq_rec.id, "-", thresholds[model_name])
        save_df = save_df.append(f_gff, ignore_index=True)
        save_df = save_df.append(r_gff, ignore_index=True)
    
    print(f"Time elapsed for {model_name} model: {round((time.time() - t) / 60, 2)} minutes")
    
    print(f"Predicted motifs count: {save_df.shape[0]} at threshold {thresholds[model_name]}")
    save_df.sort_values(by=["seqid", "start", "end"], inplace=True)
    print("Saving GFF")
    save_df.to_csv(os.path.abspath(f"{save_dir}/predicted_promoters_sk_{model_name}.gff"),
                   sep="\t", header=False, index=False)

==> Training RandomForest model
===> Predicting for: NC_000913.3 using RandomForest model
Time elapsed for RandomForest model: 0.48 minutes
Predicted motifs count: 2535 at threshold 0.1
Saving GFF
==> Training GradientBoosting model
===> Predicting for: NC_000913.3 using GradientBoosting model
Time elapsed for GradientBoosting model: 1.45 minutes
Predicted motifs count: 19687 at threshold 0.05
Saving GFF
==> Training AdaBoost model
===> Predicting for: NC_000913.3 using AdaBoost model
Time elapsed for AdaBoost model: 4.85 minutes
Predicted motifs count: 2482 at threshold 0.43
Saving GFF
==> Training MultiLayerPerceptron model
===> Predicting for: NC_000913.3 using MultiLayerPerceptron model
Time elapsed for MultiLayerPerceptron model: 0.78 minutes
Predicted motifs count: 3165 at threshold 0.005
Saving GFF
